In [ ]:
!pip install torchtext==0.6.0
!wget https://www.cl.uni-heidelberg.de/statnlpgroup/decoco/ms_coco_parallel.tar.bz2
!gunzip *.bz2
!bunzip2 ms_coco_parallel.tar.bz2
!tar -xvf *.tar
%cd ms_coco_parallel/


# !cat dev.en devtest.en test.en > en_data.txt
# !cat dev.de devtest.de test.de > de_data.txt


     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 1.2MB 10.0MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1
--2021-01-22 17:25:16--  https://www.cl.uni-heidelberg.de/statnlpgroup/decoco/ms_coco_parallel.tar.bz2
Resolving www.cl.uni-heidelberg.de (www.cl.uni-heidelberg.de)... 147.142.207.78
Connecting to www.cl.uni-heidelberg.de (www.cl.uni-heidelberg.de)|147.142.207.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31702 (31K) [application/x-bzip2]
Saving to: ‘ms_coco_parallel.tar.bz2’

ms_coco_parallel.ta 100%[===================>]  30.96K  --.-KB/s    in 0.1s    

2021-01-22 17:25:17 (220 KB/s) - ‘ms_coco_parallel.tar.bz2’ saved [31702/31702]

gzip: ms_coco_parallel.tar.bz2: unknown suffix -- ignored
ms_coco_parallel/
ms_coco_parallel/devtest.de
ms_coco_parallel/terms_of_use.txt
ms_coco_parallel/devtest.en
ms_coco_parallel/dev.en

In [ ]:
!ls

dev.de	devtest.de  README.txt	      test.de
dev.en	devtest.en  terms_of_use.txt  test.en


In [ ]:
list_train_de=[]
filepath="dev.de"
with open(filepath, encoding="utf8") as fp:  
   for line in fp:
       list_train_de.extend(line.strip().split(', '))

list_train_en=[]
filepath="dev.en"
with open(filepath, encoding="utf8") as fp:  
   for line in fp:
       list_train_en.extend(line.strip().split(', '))

list_test_de=[]
filepath="test.de"
with open(filepath, encoding="utf8") as fp:  
   for line in fp:
       list_test_de.extend(line.strip().split(', '))

list_test_en=[]
filepath="test.en"
with open(filepath, encoding="utf8") as fp:  
   for line in fp:
       list_test_en.extend(line.strip().split(', '))



In [ ]:
# Lets play with csv and text
# try to convert text/arrays to csv and make it our code firendly!!!
# Source = English
# Target = German / Dutch
import pandas as pd
train = pd.DataFrame(list(zip(list_train_en, list_train_de)), 
               columns =['trg', 'src'])

test =  pd.DataFrame(list(zip(list_test_en, list_test_de)), 
               columns =['trg', 'src'])

In [ ]:
train.head

<bound method NDFrame.head of                                                    trg                                                src
0                Giraffes in the forest on a sunny day           Giraffen an einem sonnigen Tag im Busch.
1    Zebra and water buffalo walk along the tall gr...  Ein Zebra und ein Wasserbüffel gehen neben hoh...
2    The man is only a few inches shorter than his ...  Der Mann ist nur wenige Zoll kleiner als sein ...
3                A boy and a dog running down a beach.  Ein Junge und ein Hund rennen einen Strand ent...
4    Person with long hair and a fedora standing in...  Eine Person mit langen Haaren und einer Fedora...
..                                                 ...                                                ...
261  A person holding up a GPS guide showing the sc...  Eine Hand mit einem Wurstbrötchen und einer we...
262         A man on a surfboard who is riding a wave.  Eine Gruppe von Leuten mit Regenschirmen nahe ...
263        A ted

In [ ]:
test.head

<bound method NDFrame.head of                                                    trg                                                src
0                A guy does a trick on his skateboard.  Ein Mann führt auf einen Skateboard einen Tric...
1         a dog in a field with a frisbee in its mouth  Ein Hund auf einer Wiese mit einem Frisbee im ...
2         A bag sitting next to luggage on the ground.  Eine Tasche steht neben anderem Gepäck auf dem...
3          A group of people standing around a machine  Eine Gruppe von Leuten steht neben einer Masch...
4    An elegant table full of blue arrangements of ...       Ein eleganter Tisch mit vielen blauen Vasen.
..                                                 ...                                                ...
532  A man with catchers gear on sitting in front o...  Ein Teddybär in einem Hut auf der Straße einer...
533  A woman is skiing down a snowy slope with moun...  Zwei Männer spielen ein Computerspiel und eine...
534  A tiger str

In [ ]:

train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)


In [ ]:
!ls

dev.de	devtest.de  README.txt	      test.csv	test.en
dev.en	devtest.en  terms_of_use.txt  test.de	train.csv


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import spacy
from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint
import pandas as pd
from torchtext.data import Field, BucketIterator, TabularDataset
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from torchtext.data import Field, BucketIterator

!python -m spacy download en
!python -m spacy download de


spacy_eng = spacy.load("en")
spacy_ger = spacy.load("de")


def tokenize_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]


def tokenize_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]


english = Field(sequential=True, use_vocab=True, tokenize=tokenize_eng, lower=True)
german = Field(sequential=True, use_vocab=True, tokenize=tokenize_ger, lower=True)

fields = {"trg": ("trg", english), "src": ("src", german)}

train_data, test_data = TabularDataset.splits(
    path="", train="train.csv", test="test.csv", format="csv", fields=fields
)

english.build_vocab(train_data, max_size=10000, min_freq=2)
german.build_vocab(train_data, max_size=10000, min_freq=2)

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), batch_size=32, device="cuda"
)

for batch in train_iterator:
    print(batch)

# string to integer (stoi)
print(f'Index of the word (the) is: {english.vocab.stoi["the"]}')

# print integer to string (itos)
print(f"Word of the index (16) is: {english.vocab.itos[16]}")
print(f"Word of the index (17) is: {english.vocab.itos[17]}")
print(f"Word of the index (18) is: {english.vocab.itos[18]}")
print(f"Word of the index (19) is: {english.vocab.itos[19]}")
print(f"Word of the index (20) is: {english.vocab.itos[20]}")
print(f"Word of the index (21) is: {english.vocab.itos[21]}")
print(f"Word of the index (0) is: {english.vocab.itos[0]}")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 1.2MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=ea5348f691bc65c906b16e435358108c290016ddb00e0a95aeb5d1c0f312d275
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhsglppz/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')

[torchtext.data.batch.Batch of size 10]
	[.trg]:[t

In [1]:
!pip install torchtext==0.6.0
import torch
from torchtext.data.metrics import bleu_score
import spacy



def translate_sentence(model, sentence, german, english, device, max_length=50):
    # Load german tokenizer
    spacy_ger = spacy.load("de")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]

def bleu(data, model, german, english, device):
    targets= []
    outputs= []

    for eg in data:
        src = vars(eg)['src']
        trg = vars(eg)['trg']

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1] # eos removed

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 1.2MB 7.6MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [4]:
# Senetence : Giraffen an einem sonnigen Tag im Busch.
# Converted Expected : Giraffes in the forest on a sunny day

In [ ]:

class Transformer(nn.Module):
    def __init__(self,
                embedding_size,
                src_vocab_size,
                trg_vocab_size,
                src_pad_idx,
                num_heads,
                num_encoder_layers,
                num_decoder_layers,
                forward_expansion,
                dropout,
                max_len,
                device
    ):
        super(Transformer,self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size,embedding_size)
        self.src_positional_embedding = nn.Embedding(max_len,embedding_size)
        self.trg_word_embedding =  nn.Embedding(trg_vocab_size,embedding_size)
        self.trg_positional_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
                    embedding_size,
                    num_heads,
                    num_encoder_layers,
                    num_decoder_layers,
                    forward_expansion,
                    dropout
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_masks(self,src):
        # src : (src_len, bs)
        src_mask = src.transpose(0,1) == self.src_pad_idx
        # (bs, src_len)
        return src_mask

    def forward(self, src, trg):
        src_seq_len , N = src.shape
        trg_seq_len , N = trg.shape

        src_positions =(
            torch.arange(0, src_seq_len).unsqueeze(1).expand(src_seq_len, N).to(self.device)
        )
        trg_positions = (
            torch.arange(0, trg_seq_len).unsqueeze(1).expand(trg_seq_len, N).to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_positional_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_positional_embedding(trg_positions))
        )
        src_padding_mask = self.make_src_masks(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_len).to(self.device)

        out = self.transformer(embed_src, embed_trg, src_key_padding_mask=src_padding_mask,tgt_mask=trg_mask)

        out = self.fc_out(out)
        return out

#
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model =True
save_model = False

#training hyperparameters
num_epochs = 100
learning_rate = 3e-4
batch_size= 32

#model hyperparameters
src_vocab_size = len(german.vocab)
trg_vocab_size = len(english.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.1
max_len = 100
forward_expansion = 4
src_pad_idx = english.vocab.stoi['<pad>']

writer = SummaryWriter("runs/loss_plot")
step = 0

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
        (train_data, validation_data, test_data),
        batch_size = batch_size,
        sort_within_batch=True,
        sort_key = lambda x: len(x.src),
        device = device)

model = Transformer(embedding_size,
            src_vocab_size,
            trg_vocab_size,
            src_pad_idx,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
            max_len,
            device).to(device)

optimizer = optim.Adam(model.parameters(),lr = learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

'''if load_model:
    load_checkpoint(torch.load('my_checkpoint.pth.tar',map_location=torch.device('cpu')), model, optimizer)
    print("Loaded")'''

train = True
if train:
    for epoch in range(num_epochs):
        print(f'[Epoch{epoch+1}/{num_epochs}]')

        if save_model:
            checkpoint= {
            "state_dict" : model.state_dict(),
            "optimizer" : optimizer.state_dict(),
            }
        #save_checkpoint(checkpoint)

        for batch_idx,batch in enumerate(train_iterator):
            inp_data = batch.src.to(device)
            target = batch.trg.to(device)

            #forward prop
            output = model(inp_data, target[:-1])

            output = output.reshape(-1,output.shape[2])
            target = target[1:].reshape(-1)

            optimizer.zero_grad()

            loss = criterion(output, target)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm =1)

            optimizer.step()

            writer.add_scalar("Training loss", loss, global_step=step)
            step+=1
sen1 = "Giraffen an einem sonnigen Tag im Busch."

a = translate_sentence(model, sen1, german,english, device, max_length=30)

print("English")
print(a)


import sys
sys.argv=['']
del sys

score = bleu(test_data[1:150],model,german,english,device)
print("score:")
print(score)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 19.3MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=146ba67e58986f3bdd0ea310c14a9262ec7ecd9bb7235f88ec13df13a4bc1fd6
  Stored in directory: /tmp/pip-ephem-wheel-cache-qv2be5wf/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 965kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 277kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 265kB/s]


[Epoch1/100]
[Epoch2/100]
[Epoch3/100]
[Epoch4/100]
[Epoch5/100]
[Epoch6/100]
[Epoch7/100]
[Epoch8/100]
[Epoch9/100]
[Epoch10/100]
[Epoch11/100]
[Epoch12/100]
[Epoch13/100]
[Epoch14/100]
[Epoch15/100]
[Epoch16/100]
[Epoch17/100]
[Epoch18/100]
[Epoch19/100]
[Epoch20/100]
[Epoch21/100]
[Epoch22/100]
[Epoch23/100]
[Epoch24/100]
[Epoch25/100]
[Epoch26/100]
[Epoch27/100]
[Epoch28/100]
[Epoch29/100]
[Epoch30/100]
[Epoch31/100]
[Epoch32/100]
[Epoch33/100]
[Epoch34/100]
[Epoch35/100]
[Epoch36/100]
[Epoch37/100]
[Epoch38/100]
[Epoch39/100]
[Epoch40/100]
[Epoch41/100]
[Epoch42/100]
[Epoch43/100]
[Epoch44/100]
[Epoch45/100]
[Epoch46/100]
[Epoch47/100]
[Epoch48/100]
[Epoch49/100]
[Epoch50/100]
[Epoch51/100]
[Epoch52/100]
[Epoch53/100]
[Epoch54/100]
[Epoch55/100]
[Epoch56/100]
[Epoch57/100]
[Epoch58/100]
[Epoch59/100]
[Epoch60/100]
[Epoch61/100]
[Epoch62/100]
[Epoch63/100]
[Epoch64/100]
[Epoch65/100]
[Epoch66/100]
[Epoch67/100]
[Epoch68/100]
[Epoch69/100]
[Epoch70/100]
[Epoch71/100]
[Epoch72/100]
[

In [ ]:
sen1 = "Alle die wandern sind nicht verloren"
# English Original: All those who wander are not lost
a = translate_sentence(model, sen1, german,english, device, max_length=30)

print("English")
print(a)


English
['all', 'the', 'back', 'of', 'the', 'skate', '.', '<eos>']
